In [1]:
import pandas as pd
import numpy as np
import cv2

In [2]:
df = pd.read_pickle("/home/sabrine/social_tracking/syncPresentation")

In [3]:
len(df)

216

In [4]:
df.iloc[215]

seq                                                        2462
secs                                                 1493379758
nsecs                                                 587131586
tracks        \n  - \n    id: 11\n    x: 2.20321823795\n    ...
time                                        1493379758587131586
frameMatch                                  1493379757083536490
Name: 215, dtype: object

In [5]:
df.iloc[215].tracks

'\n  - \n    id: 11\n    x: 2.20321823795\n    y: 0.0574748283926\n    height: 1.62710236584\n    distance: 2.17158484459\n    age: 11.773777\n    confidence: 1.86164275344\n    visibility: 0\n    box_2D: \n      x: 178\n      y: 49\n      width: 134\n      height: 269\n \n'

In [6]:
file = open("/home/sabrine/notebook/framesRGB/1493379757083536490.jpg")

In [7]:
img = cv2.imread("/home/sabrine/notebook/framesRGB/1493379757083536490.jpg")

In [8]:
img.shape

(540, 960, 3)

In [17]:
cv2.imshow('', img)
cv2.waitKey()
cv2.destroyAllWindows()

In [24]:
x=178
y=49
width=134
height=269

In [25]:
rectangle = cv2.rectangle(img,(x,y),(x + width,y + height),(0,255,0),3)

In [26]:
cv2.imshow('', rectangle)
cv2.waitKey()
cv2.destroyAllWindows()

In [15]:
shaped = img[178 - 20 : 178 + 20][49 - 20: 49 + 20][:]

In [16]:
cv2.imshow('', shaped)
cv2.waitKey()
cv2.destroyAllWindows()